# [모듈 2.1] 모델 훈련 스텝 및 모델 등록 스텝 개발 
## 부제목: SageMaker 모델 빌드 파이프라인을 이용한 모델 빌드 오케스트레이션

이 노트북은 "모델 훈련" 및 "모델 등록" 의 2 가지 스텝을 정의하고, 모델 빌딩 파이프라인을 생성하여 실행하는 노트북 입니다.
아래의 목차와 같이 노트북 실행이 될 예정이고
전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 1. 모델 훈련 및 모델 등록 개요 
- 2. 기본 라이브러리 로딩
- 3. 훈련에 사용할 전처리된 파일을 확인
- 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 6. 모델 레지스트리에서 모델 등록 확인
- 7. 훈련 아티펙트 경로 추출

---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 1. 모델 훈련 및 모델 등록 개요
- 모델 훈련 및 등록 단계의 개발자 가이드 
    - [모델 훈련 스텝](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training)
    - [모델 등록 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model)
    - [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)



# 2. 기본 라이브러리 로딩

세이지 메이커 관련 라이브러리를 로딩 합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## 2.1 노트북 변수 로딩


저장된 변수를 확인 합니다.

In [2]:
%store

Stored variables and their in-db values:
bucket                             -> 'sagemaker-ap-northeast-2-242201274000'
claims_data_uri                    -> 's3://sagemaker-ap-northeast-2-242201274000/sagema
customers_data_uri                 -> 's3://sagemaker-ap-northeast-2-242201274000/sagema
input_data_uri                     -> 's3://sagemaker-ap-northeast-2-242201274000/sagema
input_preproc_data_uri             -> 's3://sagemaker-ap-northeast-2-242201274000/sagema
project_prefix                     -> 'sagemaker-webinar-pipeline-advanced'
test_preproc_data_uri              -> 's3://sagemaker-ap-northeast-2-242201274000/sagema
train_preproc_data_uri             -> 's3://sagemaker-ap-northeast-2-242201274000/sagema


기존 노트북에서 저장한 변수를 로딩 합니다.

In [3]:
%store -r

# 3. 훈련에 사용할 전처리된 파일을 확인
이후에 훈련에서 사용할 S3의 저장된 전처리 데이터를 확인 합니다.

In [4]:
! aws s3 ls {train_preproc_data_uri} --recursive

2024-11-07 09:15:54     682602 sagemaker-webinar-pipeline-advanced/preporc/train.csv


In [5]:
train_prep_df = pd.read_csv(train_preproc_data_uri)
train_prep_df.head()

ImportError: Install s3fs to access S3

# 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성


## 4.1 모델 빌딩 파이프라인 변수 생성

파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.

지원되는 파라미터 타입은 다음과 같습니다:

* `ParameterString` - 파이썬 타입에서 `str` 
* `ParameterInteger` - 파이썬 타입에서 `int` 
* `ParameterFloat` - 파이썬 타입에서 `float` 

이들 파라미터를 정의할 때 디폴트 값을 지정할 수 있으며 파이프라인 실행시 재지정할 수도 있습니다. 지정하는 디폴트 값은 파라미터 타입과 일치하여야 합니다.

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

* `processing_instance_type` - 프로세싱 작업에서 사용할 `ml.*` 인스턴스 타입 
* `processing_instance_count` - 프로세싱 작업에서 사용할 인스턴스 개수 
* `training_instance_type` - 학습작업에서 사용할 `ml.*` 인스턴스 타입
* `model_approval_status` - 학습된 모델을 CI/CD를 목적으로 등록할 때의 승인 상태 (디폴트는 "PendingManualApproval")
* `input_data` - 입력데이터에 대한 S3 버킷 URI



파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.


In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

train_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)


train_instance_count = ParameterInteger(
    name="TrainInstanceCount",
    default_value=1
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


input_data = ParameterString(
    name="InputData",
    default_value=train_preproc_data_uri,
)


## 4.2 하이퍼파라미터 세팅

기본 XGBoost 하이퍼파라미터 외에 `scale_pos_weight` 는 레이블이 뷸균형이 되어 있을 경우에, 레이블 값의 가중치를 부여하는 파라미터 입니다. 레이블 0, 1 의 비율에 따라 지정합니다.

In [ ]:
from src.p_utils import get_pos_scale_weight
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')

In [ ]:
hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "alpha" : "0", 
        "eta": "0.3",
        "min_child_weight": "1",
        "objective": "binary:logistic",
        "num_round": "100",
}



## 4.3 Estimator 생성

Estimator 생성시에 인자가 필요 합니다. 주요한 인자만 보겠습니다.
- 사용자 훈련 코드 ""xgboost_script.py"
- 훈련이 끝난 후에 결과인 모델 아티펙트의 경로 "estimator_output_path" 지정 합니다. 지정 안할 시에는 디폴트 경로로 저장 됩니다.


In [ ]:
from sagemaker.xgboost.estimator import XGBoost

estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
print("estimator_output_path: \n", estimator_output_path)

xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    #code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = train_instance_type,
    framework_version = "1.0-1")

## 4.4 모델 훈련 스탭 생성
스텝 생성시에 위에서 생성한 Estimator 입력 및 입력 데이타로서 전처리 데이터가 존재하는 S3 경로를 제공합니다.

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="Fraud-Advance-Train",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= input_data,
            content_type="text/csv"
        ),
    },
)

## 4.5 모델 등록 스텝
RegisterModel인스턴스를 생성하기 위해 학습단계에서 선언한 Estimator 인스턴스를 사용합니다. 모델 패키지는 재사용가능한 모델 아티팩트의 추상화이며 추론을 위해 필요한 모든 정보를 포함하고 있습니다. 예를 들어, 모델의 Weight가 있는 위치나 추론이미지 등의 설정이 이에 포함됩니다.

모델 패키지 그룹은 모델 패키지의 집합입니다. 특정 머신러닝 문제를 해결하기 위해 모델 패키지 그룹을 생성하고, 새로운 버전의 모델패키지를 추가할 수 있습니다. 일반적으로 SageMaker pipeline작업이 실행될 때 마다 새로운 버전의 모델패키지를 생성하고 모델패키지그룹에 추가하게 됩니다.

RegisterModel은 Python SDK에서 Estimator 인스턴스의 register 메소드와 유사합니다.

구체적으로, TrainingStep으로 정의한 step_train에서 S3ModelArtifacts를 속성을 이용하여 모델을 전달합니다. TrainingStep의 properties에서 활용할 수 있는 속성들은 DescribeTrainingJob 의 응답 오브젝트에 대응됩니다.

본 노트북에서 사용된 모델패키지 그룹이름은 SageMaker Project와 함께 모델 레지스트리와 CI/CD 작업에서 직접 활용될 수 있습니다.

- 모델 그룹 리스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)   

### 모델 그룹 생성

- 참고
    - 모델 그룹 릭스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)
    - 모델 지표 등록: [Model Quality Metrics](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)

In [ ]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}
response = sm_client.list_model_package_groups(NameContains=model_package_group_name)
if len(response['ModelPackageGroupSummaryList']) == 0:
    print("No model group exists")
    print("Create model group")    
    
    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))    
else:
    print(f"{model_package_group_name} exitss")

모델이 모델 레지스트리에 저장될때 "모델 승인 상태" 를 "PendingManualApproval" 로 지정 합니다.

In [ ]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

In [ ]:
from sagemaker.workflow.step_collections import RegisterModel

step_register = RegisterModel(
    name= f"Fraud-Advance-RegisterModel",
    estimator=xgb_train,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,    
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)

# 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행



## 5.1 파이프라인 정의
파이프라인 정의시에 아래 3개의 인자를 제공합니다.
- 파이프라인 이름
- 파이프라인 파라미터
- 파이프라인 실험 설정
- 스텝 정의 (여기서는 훈련, 모델 등록 두가지 스텝 임)

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig


pipeline_name = project_prefix + "-Train-Register-Step"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        train_instance_type,    
        train_instance_count, 
        input_data,
        model_approval_status,
    ],
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_train, step_register],
)

## 5.2 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

## 5.3 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [ ]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

워크플로우의 실행상황을 살펴봅니다. 

In [ ]:
execution.describe()

## 5.4 파이프라인 실행 기다리기

In [ ]:
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

## 5.5 파이프라인 실행 단계 기록 보기

In [ ]:
execution.list_steps()

## 5.6 세이지 메이커 스튜디오에서 확인하기
아래의 세가지의 스크린 샷은 다음과 같습니다.
- 파이프라인의 실형 한 내역 입니다. 같은 파이프라인을 4번 실행한 것을 확인 됩니다.
- 파이프라인에 입력 파라미터로 제공된 것을 확인합니다.
- 두 개의 스텝으로 이루어진 파이프라인 그래프을 확인할 수 있습니다.

![pipeline-train-register.png](img/pipeline-train-register.png)

# 6. 모델 레지스트리에서 모델 등록 확인
위에서 등록한 모델 그룹 이름을 통해서 어떤 모델이 등록되었는지를 확인 합니다.
- 등록된 모델 버전에 대한 보기 --> [모델 버전의 세부 정보 보기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry-details.html)

## 6.1 세이지 메이커 스튜디오에서 모델 레지스트리 확인
세이지 메이커 스튜디오에 로그인해서 확인이 가능합니다.
- 아래는 "sagemaker-webinar-pipeline-advanced' 라는 모델 그룹에 4개의 모델 버전이 등록 되어 있는 것을 보여 주고 있습니다.
- Version 4 의 모델 상세 내역을 확인 합니다.

![model-registry.png](img/model-registry.png)

## 6.2 SageMaker Python SDK 로 모델 레지스트리 내용 학인 

In [ ]:
# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
response

#### 등록된 모델 버전의 상세 정보 확인

In [ ]:
ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

# 7 훈련 아티펙트 경로 추출

위의 훈련 스텝이 완료되면 실행해주세요

In [ ]:
from src.p_utils import get_train_artifact
    
train_model_artifact = get_train_artifact(execution, sm_client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


In [ ]:
image_uri = xgb_train.image_uri

훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [ ]:
%store train_model_artifact
%store image_uri